# 2. Preprocessing, Indexing, and Approximate Matching

This notebook explores and compares several methods for finding DNA patterns in genomic sequences, with a focus on both **exact** and **approximate** matching. I will evaluate the performance of basic and advanced algorithms, then introduce indexing methods to accelerate search efficiency.

### Goals:
- Compare **Naive Pattern Matching** and the **Boyer-Moore Algorithm** using character and alignment counts.
- Implement and test **K-mer Indexing** for fast exact matching.
- Explore **Subsequence Indexing** (spaced seeds) for efficient approximate matching allowing up to 2 mismatches (no indels).

I will apply these tools to a real genomic dataset, an exerpt of human chromosome 1 (`chr1.GRCh38.excerpt.fasta`), and analyze trade-offs between accuracy and performance.

I used this notebook to answer the following questions:

- Question 1: How many alignments does the naive exact matching algorithm try when matching the string 
`GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG` (derived from human Alu sequences) to the excerpt of human chromosome 1?  (Don't consider reverse complements.)
    - 799954

- Question 2: How many character comparisons does the naive exact matching algorithm try when matching the string `GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG` (derived from human Alu sequences) to the excerpt of human chromosome 1?  (Don't consider reverse complements.)
    - 799952

- Question 3: Question 3
How many alignments does Boyer-Moore try when matching the string `GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG` (derived from human Alu sequences) to the excerpt of human chromosome 1?  (Don't consider reverse complements.)
    - 127974

- Question 4: How many times does the string `GGCGCGGTGGCTCACGCCTGTAAT` which is derived from a human Alu sequence, occur with up to 2 substitutions in the excerpt of human chromosome 1?  (Don't consider reverse complements here.)
    - 19

- Question 5: How many total index hits are there when searching for occurrences of `GGCGCGGTGGCTCACGCCTGTAAT` with up to 2 substitutions in the excerpt of human chromosome 1?
    - 90

- Question 6: How many total index hits are there when searching for `GGCGCGGTGGCTCACGCCTGTAAT` with up to 2 substitutions in the excerpt of human chromosome 1?  (Again, don't consider reverse complements.)
    - 79

## Import Libraries and Data

In [3]:
from bm_preproc import BoyerMoore
from kmer_index import Index
from subseq_index import SubseqIndex

In [4]:
# Read genome from FASTA file in folder (chr1.GRCh38.excerpt.fasta) and prepare pattern matching variables

# Set filename
filename = 'chr1.GRCh38.excerpt.fasta'

def readGenome(filename):
    genome = ''
    with open(filename, 'r') as f:
        for line in f:
            # ignore header line with genome information
            if not line[0] == '>':
                genome += line.rstrip()
    return genome

# Set text 't' to the genome from file
t = readGenome(filename)

# Set pattern 'p' to desired string to match

# Patterns used in this project:
# 'ACTGTCCA' (used for Naive and Boyer-Moore)
# 'CGTAGCAGA' (used for Naive and Boyer-Moore)
# 'GGCGCGGTGGCTCACGCCTGTAAT' (length 24 for indexing methods)

p = 'ACTGTCCA'

# Preprocess pattern 'p' through Boyer-Moore preprocessing
p_bm = BoyerMoore(p)

## Naive Pattern Matching

In [5]:
# Naive Pattern Matching
# Locate index position of matches of pattern p in text t

def naive(p, t):
    """
    Do Naive Pattern Matching
    Arguments:
    p=pattern
    t=text
    """
    occurrences = []
    for i in range(len(t) - len(p) + 1):  # loop over alignments
        match = True
        for j in range(len(p)):  # loop over characters
            if t[i+j] != p[j]:   # compare characters
                match = False
                break
        if match:
            occurrences.append(i)  # all chars matched; record
    return occurrences

# Locate matches of pattern 'p' within genome 't' using Naive Pattern Matching method
# Assign output variables
occurrences = naive(p, t)

# Format results
print(f'Matches were found at indices: {occurrences}')

Matches were found at indices: [65168, 69168, 133223, 278881, 350637, 381387, 413823, 454011, 460956, 487636, 572559, 589837, 669022, 725573, 763494, 798784]


In [4]:
# Naive Pattern Matching with Counts
# Locate index position of matches of pattern p in text t

def naive_with_counts(p, t):
    """
    Do Naive Pattern Matching and count number of character comparisons performed and number of alignments tried.
    Arguments:
    p=pattern
    t=text
    """
    occurrences = []
    char_count = 0
    align_count = 0
    for i in range(len(t) - len(p) + 1):  # loop over alignments
        match = True
        align_count += 1
        for j in range(len(p)):  # loop over characters
            if t[i+j] != p[j]:   # compare characters
                match = False
                char_count += 1
                break
        if match:
            occurrences.append(i)  # all chars matched; record
    return occurrences, char_count, align_count

# Locate matches of pattern 'p' within genome 't' using Naive Pattern Matching method with counts.
# Assign output variables
occurrences, char_count, align_count = naive_with_counts(p, t)

# Format results
print(f'Matches were found at indices: {occurrences}')
print(f'Number of character comparisons performed: {char_count}')
print(f'Number of alignments tried: {align_count}')

Matches were found at indices: [56922, 262042, 364263, 657496, 717706]
Number of character comparisons performed: 799972
Number of alignments tried: 799977


## Boyer-Moore

In [5]:
# Boyer-Moore

def boyer_moore(p, p_bm, t):
    """ 
    Do Boyer-Moore matching. 
    Arguments:
    p=pattern
    t=text
    p_bm=BoyerMoore object for p
    """
    i = 0
    occurrences = []
    while i < len(t) - len(p) + 1:
        shift = 1
        mismatched = False
        for j in range(len(p)-1, -1, -1):
            if p[j] != t[i+j]:
                skip_bc = p_bm.bad_character_rule(j, t[i+j])
                skip_gs = p_bm.good_suffix_rule(j)
                shift = max(shift, skip_bc, skip_gs)
                mismatched = True
                break
        if not mismatched:
            occurrences.append(i)
            skip_gs = p_bm.match_skip()
            shift = max(shift, skip_gs)
        i += shift
    return occurrences

# Locate matches of pattern 'p' within genome 't' using Boyer-Moore method
# Assign output variables
occurrences = boyer_moore(p, p_bm, t)

# Format results
print(f'Matches were found at indices: {occurrences}')

Matches were found at indices: [56922, 262042, 364263, 657496, 717706]


In [6]:
# Boyer-Moore

def boyer_moore_with_counts(p, p_bm, t):
    """ 
    Do Boyer-Moore matching and count number of character comparisons performed and number of alignments tried.
    Arguments:
    p=pattern
    t=text
    p_bm=BoyerMoore object for p
    """
    i = 0
    occurrences = []
    char_count = 0
    align_count = 0
    while i < len(t) - len(p) + 1:
        shift = 1
        align_count += 1
        mismatched = False
        for j in range(len(p)-1, -1, -1):
            char_count += 1
            if p[j] != t[i+j]:
                skip_bc = p_bm.bad_character_rule(j, t[i+j])
                skip_gs = p_bm.good_suffix_rule(j)
                shift = max(shift, skip_bc, skip_gs)
                mismatched = True
                break
        if not mismatched:
            occurrences.append(i)
            skip_gs = p_bm.match_skip()
            shift = max(shift, skip_gs)
        i += shift
    return occurrences, char_count, align_count

# Locate matches of pattern 'p' within genome 't' using Boyer-Moore method with counts
boyer_moore(p, p_bm, t)

# Assign output variables
occurrences, char_count, align_count = boyer_moore_with_counts(p, p_bm, t)

# Format results
print(f'Matches were found at indices: {occurrences}')
print(f'Number of character comparisons performed: {char_count}')
print(f'Number of alignments tried: {align_count}')

Matches were found at indices: [56922, 262042, 364263, 657496, 717706]
Number of character comparisons performed: 196873
Number of alignments tried: 126203


## Comparison of Naive Pattern Matching and Boyer-Moore Algorithms

### p = 'ACTGTCCA'

|   Algorithm   |  Number of Matches | Character Comparisons   |   Alignments Attempted   |
|:-----------:|:------------:|:------------:|:------------:|
|   Naive Pattern Matching   | 16 | 799991   |   799992   |
|   Boyer-Moore   | 16 | 295455    |   196637   |

### p = 'CGTAGCAGA'

|   Algorithm   |  Number of Matches | Character Comparisons   |   Alignments Attempted   |
|:-----------:|:------------:|:------------:|:------------:|
|   Naive Pattern Matching   | 1 |  799977    |   799993   |
|   Boyer-Moore   | 1 |  283785    |   190295   |

### Reduction of Work Performed by Boyer-Moore versus Naive Pattern Matching

|   Pattern  |  Character Comparisons | Alignments Attempted |
|:-----------:|:------------:|:------------:|
|   `p = 'ACTGTCCA'`  | 63.1% | 75.4% |
|   `p = 'CGTAGCAGA'`  | 64.5% | 76.2% |

## *K*-mer Indexing with 2 Mismatches Allowed

In [6]:
# Implementation of pigeonhole principle using a k-mer index of genome and allowing for 2 mismatches
def approximate_match_2mm(p, t, index):
    """
    Find approximate occurrences of pattern p in text t with up to 2 mismatches
    using a k-mer index (assumes k=8).
    
    Args:
        p (str): Length-24 pattern
        t (str): Genome text
        index (Index): Index object supporting .query(kmer)
    
    Returns:
        list: Sorted list of match starting positions with ≤2 mismatches
    """
    assert len(p) == 24, "Pattern must be length 24"
    k = 8  # k-mer length used to build the index
    matches = set()
    total_hits = 0

    for i in range(3):  # split pattern into 3 overlapping 8-mers
        kmer = p[i * k:(i + 1) * k]
        hits = index.query(kmer)
        total_hits += len(hits)  # Count the number of hits for this 8-mer
        for pos in hits:
            start = pos - i * k
            if start < 0 or start + len(p) > len(t):
                continue  # skip out-of-bound matches
            window = t[start:start + len(p)]
            mismatches = sum(1 for a, b in zip(p, window) if a != b)
            if mismatches <= 2:
                matches.add(start)

    return sorted(matches), total_hits

# Create index of genome with 8-mers (k=8)
index = Index(t, 8)

# Run the function
matches, total_hits = approximate_match_2mm(p, t, index)

# Print results
print(f'Matches were found at indices: {matches}')
print(f'Number of index hits: {total_hits}')

AssertionError: Pattern must be length 24

## Subsequence Indexing with 2 Mismatches Allowed

In [7]:
# Implementation of pigeonhole principle using a subsequence index of genome and allowing for 2 mismatches
def approximate_match_subseq(p, t, subseq_index):
    assert len(p) == 24, "Pattern must be length 24"
    
    k = subseq_index.k
    ival = subseq_index.ival
    span = 1 + ival * (k - 1)

    matches = set()
    index_hits = 0

    for offset in range(ival):
        if offset + span > len(p):
            continue

        pattern_substring = p[offset : offset + span]
        hits = subseq_index.query(pattern_substring)
        index_hits += len(hits)

        for hit in hits:
            start = hit - offset
            if start < 0 or start + len(p) > len(t):
                continue
            window = t[start : start + len(p)]
            mismatches = sum(1 for a, b in zip(p, window) if a != b)
            if mismatches <= 2:
                matches.add(start)

    return sorted(matches), index_hits

# Create subsequence index with k=8 (length of subsequence) and ival=3 (spacing of characters) and run the function
subseq_index = SubseqIndex(t, 8, 3)
matches, index_hits = approximate_match_subseq(p, t, subseq_index)

# Print results
print(f'Matches were found at indices: {matches}')
print(f'Number of index hits: {total_hits}')

AssertionError: Pattern must be length 24